In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\sajee\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\en_core_web_sm
-->
C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You do not have sufficient privilege to perform this operation.


In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en_core_web_sm')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [5]:
#sentences = list(alice_doc.sents)

#toks = [token.pos_ for token in words] 

In [6]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [7]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400


,tired,connect,hurt,singleness,future,sheriff,sweetness,golden,save,confide,...,affection,disgraceful,hard,curious,honest,skurrie,solicit,peg,text_sentence,text_source
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [8]:
#creating a feature to represent the forms of speech in each sentence.
word_counts['pos'] = word_counts['text_sentence'].apply(lambda row: [token.pos_ for token in row])

In [9]:
word_counts.head()

,tired,connect,hurt,singleness,future,sheriff,sweetness,golden,save,confide,...,disgraceful,hard,curious,honest,skurrie,solicit,peg,text_sentence,text_source,pos
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,"[PROPN, VERB, VERB, PART, VERB, ADV, ADJ, ADP,..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,"[ADV, PRON, VERB, VERB, ADP, DET, ADJ, NOUN, P..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,"[ADV, VERB, NOUN, ADV, ADV, ADJ, ADP, DET, PUN..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,"[INTJ, ADJ, PUNCT]"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,"[INTJ, ADJ, PUNCT]"


In [10]:
import itertools

In [11]:
#since the forms of speech for each sentence (row) is formed in a list under one column, I'm combining all the lists into one.
l = [i for i in word_counts["pos"]]
mergedl = list(itertools.chain(*l))

In [12]:
print(mergedl)

['PROPN', 'VERB', 'VERB', 'PART', 'VERB', 'ADV', 'ADJ', 'ADP', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'VERB', 'NOUN', 'PART', 'VERB', 'PUNCT', 'ADV', 'CCONJ', 'ADV', 'PRON', 'VERB', 'VERB', 'ADP', 'DET', 'NOUN', 'DET', 'NOUN', 'VERB', 'VERB', 'PUNCT', 'CCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'PRON', 'PUNCT', 'PUNCT', 'CCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'PUNCT', 'VERB', 'PROPN', 'PUNCT', 'ADP', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'PUNCT', 'ADV', 'PRON', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'ADV', 'ADV', 'ADP', 'PRON', 'VERB', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'PRON', 'VERB', 'ADV', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'VERB', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'VERB', 'VERB', 'ADJ', 'DET', 'NOUN', 'ADP', 'VERB', 'PART', 'CCONJ', 'VERB', 'DET', 'NOUN', 'PUNCT', 'ADV', 'ADV', 'DET', 'PROPN', 'PROPN', 'ADP', 'ADJ', 'NOUN', 'VERB', 'ADV', '

In [13]:
#getting the unique values from merged list.
unique_form = list(set(mergedl))

In [14]:
print(unique_form)

['NOUN', 'ADV', 'ADP', 'VERB', 'PART', 'DET', 'PUNCT', 'CCONJ', 'PRON', 'AUX', 'INTJ', 'ADJ', 'NUM', 'PROPN']


In [19]:
#creating columns for each of the unique values in merged list. The first for loop, loops over unique forms, takes one element
#and then creates a column for that x value and the second for loop inside the list comprehension counts the number of times
#x comes up in the "pos" column.  This is done for each value in unique_form, and for each cell in the dataframe series
#word_counts["pos"]
for x in unique_form:
    word_counts[x] = [i.count(x) for i in word_counts["pos"]]
    

In [21]:
word_counts.head(20)

,tired,connect,hurt,singleness,future,sheriff,sweetness,golden,save,confide,...,PART,DET,PUNCT,CCONJ,PRON,AUX,INTJ,ADJ,NUM,PROPN
0,1,0,0,0,0,0,0,0,0,0,...,2,7,10,6,4,0,0,1,0,2
1,0,0,0,0,0,0,0,0,0,0,...,1,7,7,2,4,0,0,6,0,2
2,0,0,0,0,0,0,0,0,0,0,...,1,3,3,1,2,0,0,1,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,1,0,0,1,0,0
6,0,0,0,0,0,0,0,0,0,0,...,2,4,5,2,6,0,0,1,0,1
7,0,0,0,0,0,0,0,0,0,0,...,1,2,3,2,1,0,0,1,0,2
8,0,0,0,0,0,0,0,0,0,0,...,3,8,9,4,6,0,0,1,0,1
9,0,0,0,0,0,0,0,0,0,0,...,3,2,2,0,2,0,0,0,0,1


In [24]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split


In [27]:
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source', 'pos'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

### Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1627) (266,)
Training set score: 0.981203007518797

Test set score: 0.8595505617977528


### Original Random Forest

In [29]:

rfc = ensemble.RandomForestClassifier()

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9661654135338346

Test set score: 0.7752808988764045


### Gradient Boosting

In [30]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9699248120300752

Test set score: 0.8314606741573034


In the lesson, Logistic regression performed the best, using the original features.  The features in our dataset are continuous in their nature, so they are better suited for logistic regression because logistic regression works with the same assumptions as linear regression.  After adding new features using the pos_ attribute for NLP, Logistic Regression still performed the best, but the accuracy score did not improve.

### SVM

In [32]:
from sklearn.svm import SVC
svm = SVC(kernel = 'linear')
svm.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [34]:
from sklearn.model_selection import cross_val_score

cross_val_score(svm, X, Y, cv=10)

array([0.88888889, 0.82222222, 0.88888889, 0.86666667, 0.88888889,
       0.88636364, 0.72727273, 0.79545455, 0.72727273, 0.76744186])

Using SVM did not give an improved score.  From looking at the cross validation results, we can see that there is considerable overfitting.

# Challenge 1:

In [35]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [36]:
# Load and clean the data.  Here I'm loading 'bryant-stories.txt' from gutenberg
bstories = gutenberg.raw('bryant-stories.txt')


# The Chapter indicator is idiosyncratic
bstories = re.sub(r'Chapter \d+', '', bstories)


bstories = text_cleaner(bstories[:int(len(bstories)/10)])

In [37]:
#parse the cleaned novels
bstories_doc = nlp(bstories)

In [38]:
# Group into sentences.
bryant_sents = [[sent, "Bryant"] for sent in bstories_doc.sents]

In [41]:
# Combine the sentences from the two novels into one data frame.
sentences_newbook = pd.DataFrame(alice_sents + bryant_sents)
sentences_newbook.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [43]:


# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features_newbook(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
bryantwords = bag_of_words(bstories_doc)

# Combine bags to create a set of unique words.
common_words_newbook = set(alicewords + bryantwords)

In [44]:
# Create our data frame with features.
word_counts_newbook = bow_features_newbook(sentences_newbook, common_words_newbook)
word_counts_newbook.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450
Processing row 500
Processing row 550


,tired,POCKET,apple,father,fast,Moon,cake,hurt,whisker,stone,...,hard,curious,light,skurrie,thousand,listen,cellar,peg,text_sentence,text_source
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll


In [45]:
#creating a feature to represent the forms of speech in each sentence.
word_counts_newbook['pos'] = word_counts_newbook['text_sentence'].apply(lambda row: [token.pos_ for token in row])

In [46]:
word_counts_newbook.head()

,tired,POCKET,apple,father,fast,Moon,cake,hurt,whisker,stone,...,curious,light,skurrie,thousand,listen,cellar,peg,text_sentence,text_source,pos
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,"[PROPN, VERB, VERB, PART, VERB, ADV, ADJ, ADP,..."
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,"[ADV, PRON, VERB, VERB, ADP, DET, ADJ, NOUN, P..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,"[ADV, VERB, NOUN, ADV, ADV, ADJ, ADP, DET, PUN..."
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,"[INTJ, ADJ, PUNCT]"
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll,"[INTJ, ADJ, PUNCT]"


In [48]:
#since the forms of speech for each sentence (row) is formed in a list under one column, I'm combining all the lists into one.
newbook_poslist = [i for i in word_counts["pos"]]
merged_nb_poslist = list(itertools.chain(*newbook_poslist))

In [49]:
print(merged_nb_poslist)

['PROPN', 'VERB', 'VERB', 'PART', 'VERB', 'ADV', 'ADJ', 'ADP', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'CCONJ', 'ADP', 'VERB', 'NOUN', 'PART', 'VERB', 'PUNCT', 'ADV', 'CCONJ', 'ADV', 'PRON', 'VERB', 'VERB', 'ADP', 'DET', 'NOUN', 'DET', 'NOUN', 'VERB', 'VERB', 'PUNCT', 'CCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'PRON', 'PUNCT', 'PUNCT', 'CCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'PUNCT', 'PUNCT', 'VERB', 'PROPN', 'PUNCT', 'ADP', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'PUNCT', 'ADV', 'PRON', 'VERB', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'ADV', 'ADV', 'ADP', 'PRON', 'VERB', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'VERB', 'PRON', 'VERB', 'ADV', 'ADJ', 'CCONJ', 'ADJ', 'PUNCT', 'PUNCT', 'ADP', 'DET', 'NOUN', 'ADP', 'VERB', 'DET', 'NOUN', 'PUNCT', 'NOUN', 'VERB', 'VERB', 'ADJ', 'DET', 'NOUN', 'ADP', 'VERB', 'PART', 'CCONJ', 'VERB', 'DET', 'NOUN', 'PUNCT', 'ADV', 'ADV', 'DET', 'PROPN', 'PROPN', 'ADP', 'ADJ', 'NOUN', 'VERB', 'ADV', '

In [50]:
#getting the unique values from merged list.
unique_form_newbook = list(set(merged_nb_poslist))

In [51]:
print(unique_form_newbook)

['NOUN', 'ADV', 'ADP', 'VERB', 'PART', 'DET', 'PUNCT', 'CCONJ', 'PRON', 'AUX', 'INTJ', 'ADJ', 'NUM', 'PROPN']


In [52]:
#creating columns for each of the unique values in merged list.
for x in unique_form_newbook:
    word_counts_newbook[x] = [i.count(x) for i in word_counts_newbook["pos"]]
    

In [53]:
word_counts_newbook.head()

,tired,POCKET,apple,father,fast,Moon,cake,hurt,whisker,stone,...,PART,DET,PUNCT,CCONJ,PRON,AUX,INTJ,ADJ,NUM,PROPN
0,1,0,0,0,0,0,0,0,0,0,...,2,7,10,6,4,0,0,1,0,2
1,0,0,0,0,0,0,0,0,0,0,...,1,7,7,2,4,0,0,6,0,2
2,0,0,0,0,0,0,0,0,0,0,...,1,3,3,1,2,0,0,1,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0


In [54]:
#create new X and Y, using the new dataframe consisting of Carrol-Alice and Bryant-stories.
Y_newbook = word_counts_newbook['text_source']
X_newbook = np.array(word_counts_newbook.drop(['text_sentence','text_source', 'pos'], 1))

#create a new train and test split.
X_train_nb, X_test_nb, y_train_nb, y_test_nb = train_test_split(X_newbook, 
                                                    Y_newbook,
                                                    test_size=0.4)

### Logistic Regression with new book

In [55]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train_nb, y_train_nb)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train_nb, y_train_nb))
print('\nTest set score:', lr.score(X_test_nb, y_test_nb))

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(266, 1627) (266,)
Training set score: 0.9697885196374623

Test set score: 0.8099547511312217


### Random Forest

In [57]:
rfc = ensemble.RandomForestClassifier()

train = rfc.fit(X_train_nb, y_train_nb)

print('Training set score:', rfc.score(X_train_nb, y_train_nb))
print('\nTest set score:', rfc.score(X_test_nb, y_test_nb))

C:\Users\sajee\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Training set score: 0.9848942598187311

Test set score: 0.8009049773755657


### Gradient Boosting

In [58]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train_nb, y_train_nb)

print('Training set score:', clf.score(X_train_nb, y_train_nb))
print('\nTest set score:', clf.score(X_test_nb, y_test_nb))

Training set score: 0.9848942598187311

Test set score: 0.8235294117647058


So I replaced "austen-persuasion.txt" with "bryant-stories.txt" in the model, and then performed similar feature engineering where I used the .pos_ attribute of an NLP to create features that counted the different forms of speech present in each sentence in the data.  Gradient boosting gave me the highest score on the test set, of 82%, but there is considerable overfitting present in each of the models.